In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os, sys
import pandas as pd
import sklearn
import sklearn.svm, sklearn.discriminant_analysis, sklearn.linear_model
import time

root = '/usr/local/serenceslab/maggie/shapeDim/'

sys.path.append(os.path.join(root, 'Analysis'))
from code_utils import file_utils, data_utils
from code_utils import decoding_utils
from code_utils import stats_utils


In [2]:
debug=True; n_threads = 8

In [3]:
print('debug = %s, n_threads = %d'%(debug, n_threads))

subjects = [1]
# subjects = np.arange(1,8)
n_subjects = len(subjects)
n_rois = 11
make_time_resolved=False

# first load all data for all subjects, all tasks
maindat_all = []; repdat_all = []
mainlabs_all = []; replabs_all = []

for si, ss in enumerate(subjects):

    print('loading S%02d, main task'%ss)
    main_data, _, main_labels, roi_names = data_utils.load_main_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        # subtract mean across voxels each trial
        main_data[ri] -= np.tile(np.mean(main_data[ri], axis=1, keepdims=True), [1, main_data[ri].shape[1]])

    maindat_all += [main_data]
    mainlabs_all += [main_labels]

    print('loading S%02d, repeat task'%ss)
    rep_data, _, rep_labels, roi_names = data_utils.load_repeat_task_data(ss, make_time_resolved)

    for ri in range(n_rois):
        # subtract mean across voxels each trial
        rep_data[ri] -= np.tile(np.mean(rep_data[ri], axis=1, keepdims=True), [1, rep_data[ri].shape[1]])

    repdat_all += [rep_data]
    replabs_all += [rep_labels]

debug = True, n_threads = 8
loading S01, main task
loading S01, repeat task


In [10]:
# penalties to eval
c = 0.005


# store the decoding performance, for each dichotomy
n_tasks = 4
ccgp_acc = np.zeros((n_subjects, n_rois, n_tasks, 4))

for si, ss in enumerate(subjects):

    main_data = maindat_all[si]
    main_labels = mainlabs_all[si]
    rep_data = repdat_all[si]
    rep_labels = replabs_all[si]

    # gathering labels for main task and for repeat task.
    # all labels will be concatenated [main; repeat]
    inds_use_main = (main_labels['is_main_grid']==True) 
    inds_use_rep = (rep_labels['is_main_grid']==True) 

    center = 2.5
    # binarize these, two categories on each axis
    xlabs_main = (np.array(main_labels['ptx'])[inds_use_main]>center).astype(int)
    ylabs_main = (np.array(main_labels['pty'])[inds_use_main]>center).astype(int)
    xlabs_rep = (np.array(rep_labels['ptx'])[inds_use_rep]>center).astype(int)
    ylabs_rep = (np.array(rep_labels['pty'])[inds_use_rep]>center).astype(int)

    xlabs = np.concatenate([xlabs_main, xlabs_rep], axis=0)
    ylabs = np.concatenate([ylabs_main, ylabs_rep], axis=0)

    # repeat task is task "4" out of 4 here
    task_labs_main = np.array(main_labels['task'])[inds_use_main]
    task_labs_rep = 4 * np.ones((np.sum(inds_use_rep), ), dtype=int)
    task_labs = np.concatenate([task_labs_main, task_labs_rep], axis=0)

    is_main_task = task_labs<4

    for ri in [0]:
    # for ri in range(n_rois):

        if debug & (ri>0):
            continue
        print('proc S%02d, %s'%(ss, roi_names[ri]))

        ti = 0; tt = 1;
        
        # for ti, tt in enumerate([1,2,3,4]):

        tinds = task_labs==tt

        xlabs_task = xlabs[tinds]
        ylabs_task = ylabs[tinds]

        # data for this ROI
        dat_main = main_data[ri][inds_use_main,:][tinds[is_main_task],:]
        dat_rep = rep_data[ri][inds_use_rep,:][tinds[~is_main_task],:]
        dat = np.concatenate([dat_main, dat_rep], axis=0)

        print('processing task %d: %d total trials'%(tt, dat.shape[0]))

        xx = -1
        for split_dim, dec_dim in zip([xlabs_task, ylabs_task], [ylabs_task, xlabs_task]):

            for trni, testi in zip([0,1], [1,0]):

                print(trni, testi)
                # split the data according to this dimension
                # (decode along the other dimension)
                trninds = split_dim==trni
                tstinds = split_dim==testi

                st = time.time()

                # define model
                model = sklearn.linear_model.LogisticRegression(C = c, \
                                                                solver='lbfgs', \
                                                                penalty='l2', \
                                                                n_jobs = n_threads , \
                                                                max_iter = 1000)
                model.fit(dat[trninds,:], dec_dim[trninds])
                acc = model.score(dat[tstinds,:], dec_dim[tstinds])

                elapsed = time.time() - st
                print('acc = %.2f, elapsed = %.5f s'%(acc, elapsed))

                xx+=1
                ccgp_acc[si,ri,ti,xx] = acc

proc S01, V1
processing task 1: 384 total trials
0 1
acc = 0.62, elapsed = 0.55632 s
1 0
acc = 0.65, elapsed = 0.55862 s
0 1
acc = 0.59, elapsed = 0.56079 s
1 0
acc = 0.54, elapsed = 0.56816 s


In [11]:
ccgp_acc[si,ri,ti]

array([0.61979167, 0.64583333, 0.58854167, 0.53645833])

In [8]:
for x, y in zip([0,1], [1,0]):
    print(x,y)

0 1
1 0


In [7]:
acc

0.5364583333333334

In [5]:
dat.shape

(384, 1866)

In [2]:
subjects = np.arange(1,8)
n_subjects = len(subjects)
n_rois = 11
make_time_resolved=False
n_tasks = 4


In [3]:
ss = 1
main_data, _, main_labels, roi_names = data_utils.load_main_task_data(ss, make_time_resolved)
n_rois

11

In [4]:
for ri in range(n_rois):
    # subtract mean across voxels each trial
    main_data[ri] -= np.tile(np.mean(main_data[ri], axis=1, keepdims=True), [1, main_data[ri].shape[1]])


In [14]:
si = 0;

n_grid_pts = 16
# main_data = maindat_all[si]
# main_labels = mainlabs_all[si]

# pull out data from main grid trials only, all tasks here
inds_use = (main_labels['is_main_grid']==True) 

center = 2.5
# labels are 1-16 for grid positions
xlabs = (np.array(main_labels['ptx'])[inds_use]>center).astype(int)
ylabs = (np.array(main_labels['pty'])[inds_use]>center).astype(int)
# pt_labs = np.array([xlabs, ylabs]).T
# grid_pts, grid_labs, counts = np.unique(pt_labs, axis=0, return_inverse=True, return_counts=True)
# assert(grid_pts.shape[0]==4)

In [13]:
# leave-one-run-out
cv_labs = np.array(main_labels['run_overall'])[inds_use]
n_cv = len(np.unique(cv_labs))


In [ ]:
ri = 0;

dat = main_data[ri][inds_use,:]

c = 0.005
n_threads = 8;

for labs1 in [xlabs, ylabs]:
    for labs2 in [xlabs, ylabs]:
        
        # use the first dim as thing to train/test across
        trninds = labs1==0
        tstinds = labs1==1
        
        

In [ ]:
# using a fixed regularizing strength here
c = 0.005
n_threads = 8;

acc_each_dich = np.zeros((n_dich,n_tasks))

for di in range(n_dich):
    
    if debug and (di>1):
        continue
    
    # create binarized labels for this dichotomy
    d = dich[di,:]
    pts1 = np.arange(n_pts)[d==1]
    print('grouping:')
    print(pts1)
    labels_dich = np.isin(grid_labs, pts1).astype(int)
    assert(np.mean(labels_dich)==1/2)
    
    st = time.time()
    
    # do the decoding here
    # set up cross-validation, leave-one-run-out
    # using this generator is faster than manually looping
    cv_obj = sklearn.model_selection.LeaveOneGroupOut()
    cv_generator = cv_obj.split(dat, labels_dich, cv_labs)
    # define model
    model = sklearn.linear_model.LogisticRegressionCV(Cs = [c], \
                                                      cv = cv_generator, \
                                                    solver='lbfgs', \
                                                    penalty='l2', \
                                                    n_jobs = n_threads , \
                                                    max_iter = 1000)
    model.fit(dat, labels_dich)
    acc = np.mean(model.scores_[1])
    
    elapsed = time.time() - st
    print('acc = %.2f, elapsed = %.5f s'%(acc, elapsed))
    
    acc_each_dich[di, ti] = acc

grouping:
[0 1 2 3 4 5 6 7]
acc = 0.74, elapsed = 0.96084 s
grouping:
[0 1 2 3 4 5 6 8]
acc = 0.70, elapsed = 0.94823 s


In [ ]:
n_dich

6435

In [ ]:
# using a fixed regularizing strength here
c = 0.005
n_threads = 8;

n_conds_leaveout = 1

acc_each_dich = np.zeros((n_dich,n_tasks))

for di in range(n_dich):
    
    if debug and (di>1):
        continue
    
    # create binarized labels for this dichotomy
    d = dich[di,:]
    pts1 = np.arange(n_pts)[d==1]
    pts2 = np.arange(n_pts)[d==0]
    print('grouping:')
    print(pts1, pts2)
    labels_dich = np.isin(grid_labs, pts1).astype(int)
    assert(np.mean(labels_dich)==1/2)


grouping:
[0 1 2 3 4 5 6 7] [ 8  9 10 11 12 13 14 15]
grouping:
[0 1 2 3 4 5 6 8] [ 7  9 10 11 12 13 14 15]
